<a href="https://colab.research.google.com/github/MuhammadShifa/Horovod-with-Keras/blob/main/Horovord_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install horovod

     |████████████████████████████████| 3.2MB 5.1MB/s 
  Created wheel for horovod: filename=horovod-0.21.0-cp36-cp36m-linux_x86_64.whl size=21618615 sha256=da08c44ce4a650fdf36cdd3af316d0d5b61dae7389dff300f32d0d16ccb57206
  Stored in directory: /root/.cache/pip/wheels/ad/0c/ea/6bd706effa600df0cc11074f75e5fd8dd2d66a979cb4ae5219
Successfully built horovod


In [ ]:
tf.__version__

'2.4.0'

## Horovod with Keras MNIST Example

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import math
import tensorflow as tf
import horovod.keras as hvd

# Horovod: initialize Horovod.
hvd.init()

# Horovod: pin GPU to be used to process local rank (one GPU per process), this code because we have tf v2
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')


batch_size = 128
num_classes = 10

# Horovod: adjust number of epochs based on number of GPUs.
epochs = int(math.ceil(12.0 / hvd.size()))

# Input image dimensions
img_rows, img_cols = 28, 28

# The data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Horovod: adjust learning rate based on number of GPUs.
opt = keras.optimizers.Adadelta(1.0 * hvd.size())

# Horovod: add Horovod Distributed Optimizer.
opt = hvd.DistributedOptimizer(opt)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),
]

# Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
if hvd.rank() == 0:
    callbacks.append(keras.callbacks.ModelCheckpoint('./checkpoint-{epoch}.h5'))

model.fit(x_train, y_train,
          batch_size=batch_size,
          callbacks=callbacks,
          epochs=epochs,
          verbose=1 if hvd.rank() == 0 else 0,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/12
469/469 [==============================] - 13s 10ms/step - loss: 0.5446 - accuracy: 0.8282 - val_loss: 0.0607 - val_accuracy: 0.9809
Epoch 2/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0966 - accuracy: 0.9710 - val_loss: 0.0426 - val_accuracy: 0.9867
Epoch 3/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0691 - accuracy: 0.9790 - val_loss: 0.0310 - val_accuracy: 0.9893
Epoch 4/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0525 - accuracy: 0.9846 - val_loss: 0.0298 - val_accuracy: 0.9898
Epoch 5/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0459 - accuracy: 0.9862 - val_loss: 0.0264 - val_accuracy: 0.9912
Epoch 6/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0424 - accuracy: 0.9881 - val_loss: 0.0283 - val_accuracy: 0.9906
Epoch 7/

## Learning Step by Step Horovod

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import math
import tensorflow as tf
import horovod.keras as hvd

In [ ]:
# 1. Horovod: initialize Horovod.
hvd.init()
print(hvd.init())

In [ ]:
# 2. Horovod: pin GPU to be used to process local rank (one GPU per process), this code because we have tf v2
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

print("The gpus is: ",gpus)
print("The local rank is: ", hvd.local_rank())

The gpus is:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
The local rank is:  0


In [ ]:
batch_size = 128
num_classes = 10

# Horovod: adjust number of epochs based on number of GPUs.
epochs = int(math.ceil(12.0 / hvd.size()))
print("The size of hvd is: ", hvd.size())  #A function that returns the number of Horovod processes.
print("The no of epoch is: ",epochs)

# Input image dimensions
img_rows, img_cols = 28, 28

# The data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


The size of hvd is:  1
The no of epoch is:  12
11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
# 3. Horovod: adjust learning rate based on number of GPUs.
opt = keras.optimizers.Adadelta(1.0 * hvd.size())

# Horovod: add Horovod Distributed Optimizer.
opt = hvd.DistributedOptimizer(opt)
print("The hvd distributed optimizer is: ", opt)

The hvd distributed optimizer is:  <horovod._keras.Adadelta object at 0x7f37577a1470>


In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])
# 4. Synchronize state accross workers
callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),
]

print("The callbacks is: ", callbacks)

The callbacks is:  [<horovod.keras.callbacks.BroadcastGlobalVariablesCallback object at 0x7f37577a1390>]


In [ ]:
# 5. Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
if hvd.rank() == 0:
    callbacks.append(keras.callbacks.ModelCheckpoint('./checkpoint-{epoch}.h5'))

model.fit(x_train, y_train,
          batch_size=batch_size,
          callbacks=callbacks,
          epochs=epochs,
          verbose=1 if hvd.rank() == 0 else 0,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
469/469 [==============================] - 13s 10ms/step - loss: 0.5437 - accuracy: 0.8289 - val_loss: 0.0575 - val_accuracy: 0.9810
Epoch 2/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0999 - accuracy: 0.9705 - val_loss: 0.0423 - val_accuracy: 0.9855
Epoch 3/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0660 - accuracy: 0.9808 - val_loss: 0.0330 - val_accuracy: 0.9888
Epoch 4/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0580 - accuracy: 0.9830 - val_loss: 0.0323 - val_accuracy: 0.9893
Epoch 5/12
469/469 [==============================] - 4s 9ms/step - loss: 0.0471 - accuracy: 0.9856 - val_loss: 0.0305 - val_accuracy: 0.9889
Epoch 6/12
469/469 [==============================] - 4s 9ms/step - loss: 0.0426 - accuracy: 0.9876 - val_loss: 0.0288 - val_accuracy: 0.9900
Epoch 7/12
469/469 [==============================] - 4s 9ms/step - loss: 0.0380 - accuracy: 0.9887 - val_loss: 0.0306 - val_accuracy: 0.9899
Epoc